In [2]:
import numpy as np
from matplotlib import pyplot as plt
import os
from scipy.linalg import block_diag

In [3]:
#Mode shapes
data4 = np.load(r'C:\Users\alasm\Masteroppgave\HAR_INT\FEM\mode4_data.npz')
data15 = np.load(r'C:\Users\alasm\Masteroppgave\HAR_INT\FEM\mode15_data.npz')

x_4 = data4['x']
mode_4 = data4['mode']

x_15 = data15['x']
mode_15 = data15['mode']

#Number of nodes per bridge
nodes = len(x_4)
zero = np.zeros(nodes)

#Mode shapes for each dof along both bridges
phi_4a_z1 = np.hstack([mode_4[:, 2], zero])
phi_4a_theta1 = np.hstack([mode_4[:, 3], zero])
phi_4a_z2 = np.hstack([zero, zero])
phi_4a_theta2 = np.hstack([zero, zero])

phi_15a_z1 = np.hstack([mode_15[:, 2], zero])
phi_15a_theta1 = np.hstack([mode_15[:, 3], zero])
phi_15a_z2 = np.hstack([zero, zero])
phi_15a_theta2 = np.hstack([zero, zero])

phi_4b_z1 = np.hstack([zero, zero])
phi_4b_theta1 = np.hstack([zero, zero])
phi_4b_z2 = np.hstack([zero, mode_4[:, 2]])
phi_4b_theta2 = np.hstack([zero, mode_4[:, 3]])

phi_15b_z1 = np.hstack([zero, zero])
phi_15b_theta1 = np.hstack([zero, zero])
phi_15b_z2 = np.hstack([zero, mode_15[:, 2]])
phi_15b_theta2 = np.hstack([zero, mode_15[:, 3]])

#Mode shapes for each dof along one bridge
phi_4single_z1 = mode_4[:, 2]
phi_4single_theta1 = mode_4[:, 3]

phi_15single_z1 = mode_15[:, 2]
phi_15single_theta1 = mode_15[:, 3]

#Mode shapes for all dofs along both bridges
phi_4a = np.hstack([phi_4a_z1, phi_4a_theta1, phi_4a_z2, phi_4a_theta2])
phi_15a = np.hstack([phi_15a_z1, phi_15a_theta1, phi_15a_z2, phi_15a_theta2])
phi_4b = np.hstack([phi_4b_z1, phi_4b_theta1, phi_4b_z2, phi_4b_theta2])
phi_15b = np.hstack([phi_15b_z1, phi_15b_theta1, phi_15b_z2, phi_15b_theta2])

#Mode shape matrix for all modes
phi = np.column_stack((phi_4a, phi_15a, phi_4b, phi_15b))

#Mode shapes for all dofs along one bridge
phi_4single = np.hstack([phi_4single_z1, phi_4single_theta1])
phi_15single = np.hstack([phi_15single_z1, phi_15single_theta1])

#Mode shape matrix for all modes along one bridge
phi_single = np.column_stack((phi_4single, phi_15single))

In [4]:
#Generalizing the aerodynamic stiffness and damping matrices
dx_array = np.zeros_like(x_4)
dx_array[1:-1] = (x_4[2:] - x_4[:-2]) / 2
dx_array[0] = (x_4[1] - x_4[0]) / 2
dx_array[-1] = (x_4[-1] - x_4[-2]) / 2

dx4x4 = np.hstack([dx_array, dx_array, dx_array, dx_array, dx_array, dx_array, dx_array, dx_array])
dx2x2 = np.hstack([dx_array, dx_array])

Kae_1D_gen = np.zeros((4, 4))
Kae_2D_gen = np.zeros((4, 4))
Kae_3D_gen = np.zeros((4, 4))
Kae_4D_gen = np.zeros((4, 4))
Kae_5D_gen = np.zeros((4, 4))
Kae_Single_gen = np.zeros((2, 2))

Cae_1D_gen = np.zeros((4, 4))
Cae_2D_gen = np.zeros((4, 4))
Cae_3D_gen = np.zeros((4, 4))
Cae_4D_gen = np.zeros((4, 4))
Cae_5D_gen = np.zeros((4, 4))
Cae_Single_gen = np.zeros((2, 2))

for i in range(4):
    for j in range(4):
        integrand = phi[:, i] * phi[:, j] * dx4x4
        integral = np.sum(integrand)
        Kae_1D_gen[i, j] = np.trace(Kae_1D) * integral
        Kae_2D_gen[i, j] = np.trace(Kae_2D) * integral
        Kae_3D_gen[i, j] = np.trace(Kae_3D) * integral
        Kae_4D_gen[i, j] = np.trace(Kae_4D) * integral
        Kae_5D_gen[i, j] = np.trace(Kae_5D) * integral
        Cae_1D_gen[i, j] = np.trace(Cae_1D) * integral
        Cae_2D_gen[i, j] = np.trace(Cae_2D) * integral
        Cae_3D_gen[i, j] = np.trace(Cae_3D) * integral
        Cae_4D_gen[i, j] = np.trace(Cae_4D) * integral
        Cae_5D_gen[i, j] = np.trace(Cae_5D) * integral

for i in range(2):
    for j in range(2):
        integrand = phi_single[:, i] * phi_single[:, j] * dx2x2
        integral = np.sum(integrand)
        Kae_Single_gen[i, j] = np.trace(Kae_Single) * integral
        Cae_Single_gen[i, j] = np.trace(Cae_Single) * integral

NameError: name 'Kae_1D' is not defined

In [ ]:
coord_1 = np.zeros((nodes, 2))
coord_1[:, 0] = x_4
coord_1[:, 1] = np.ones(nodes) * (B/2 + D/2)

coord_2 = np.zeros((nodes, 2))
coord_2[:, 0] = x_4
coord_2[:, 1] = np.ones(nodes) * (-B/2 - D/2)

coord = np.vstack([coord_1, coord_2])

dxdx_ = np.abs(coord[:, np.newaxis, :] - coord[np.newaxis, :, :])


In [ ]:
blocks = [Kae_1D * c for c in dx2x2]
Kae_1D_stacked = block_diag(*blocks)

phi_4a_ny = np.column_stack([phi_4a_z1, phi_4a_theta1, phi_4a_z2, phi_4a_theta2]).flatten()
phi_15a_ny = np.column_stack([phi_15a_z1, phi_15a_theta1, phi_15a_z2, phi_15a_theta2]).flatten()
phi_4b_ny = np.column_stack([phi_4b_z1, phi_4b_theta1, phi_4b_z2, phi_4b_theta2]).flatten()
phi_15b_ny = np.column_stack([phi_15b_z1, phi_15b_theta1, phi_15b_z2, phi_15b_theta2]).flatten()

phi_ny = np.vstack([phi_4a_ny, phi_15a_ny, phi_4b_ny, phi_15b_ny])

print(phi_ny.shape, Kae_1D_stacked.shape)

Kae_1D_gen_ny = phi_ny @ Kae_1D_stacked @ phi_ny.T
print(Kae_1D_gen_ny)
print(Kae_1D_gen)

In [ ]:

def plot_response_spectrum(node):
    
    plt.figure(figsize=(10, 6))
    plt.plot(omega, S_r_1D[:, node, node], label="1D", color='blue')
    plt.plot(omega, S_r_2D[:, node, node], label="2D", color='orange')
    plt.plot(omega, S_r_3D[:, node, node], label="3D", color='green')
    plt.plot(omega, S_r_4D[:, node, node], label="4D", color='red')
    plt.plot(omega, S_r_5D[:, node, node], label="5D", color='purple')
    if node == 36:
        plt.plot(omega, S_r_Single[:, node, node], label="Single", color='brown')
    elif node == 167:
        plt.plot(omega, S_r_Single[:, 136, 33+67], label="Single", color='brown')
    elif node == 368:
        plt.plot(omega, S_r_Single[:, 33, 33], label="Single", color='brown')
    elif node == 502:
        plt.plot(omega, S_r_Single[:, 33+67, 33+67], label="Single", color='brown')
    plt.xlabel("Frequency ω [rad/s]")
    plt.ylabel("Response Spectrum")
    plt.legend()
    plt.grid(True)
    plt.xscale('log')
    plt.yscale('log')
    plt.show()

plot_response_spectrum(i1)
plot_response_spectrum(i2)
plot_response_spectrum(i3)
plot_response_spectrum(i4)

In [ ]:
print(nodes)
diag = np.diagonal(S_r_1D, axis1=1, axis2=2)
diag_real = np.real(diag)
max_per_dof = np.max(diag_real, axis=0)

max_per_dof = max_per_dof[::4]
print(max_per_dof)
print(np.argmax(max_per_dof))

"""print(max_per_dof[70])
#print(max_per_dof)
print(max_per_dof.shape)

i1 = np.argmax(max_per_dof[0:67])
i2 = np.argmax(max_per_dof[135:202]) + 135
i3 = np.argmax(max_per_dof[335:402]) + 335
i4 = np.argmax(max_per_dof[470:535]) + 470

print(i1, i2, i3, i4)
print(max_per_dof[i1])
print(max_per_dof[i2])
print(max_per_dof[i3])
print(max_per_dof[i4])"""